# Data Challenge: Smart meter is coming
by BCM Energy - Planète OUI

In [243]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.layers import LSTM, Dense, Flatten
from keras.models import Sequential

from sklearn.base import BaseEstimator, TransformerMixin

import datetime

In [244]:
from platform import python_version

print(python_version())

3.7.0


In [245]:
x_train = pd.read_csv('provided_data_and_metric/X_train_6GWGSxz.csv')
y_train = pd.read_csv('provided_data_and_metric/y_train_2G60rOL.csv')
x_test = pd.read_csv('provided_data_and_metric/X_test_c2uBt2s.csv')

## Data Exploration

In [195]:
x_train.head()

,time_step,consumption,visibility,temperature,humidity,humidex,windchill,wind,pressure,Unnamed: 9
0,2013-03-17T00:01:00.0,550.4000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-03-17T00:02:00.0,548.6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-03-17T00:03:00.0,549.3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-03-17T00:04:00.0,549.3667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-03-17T00:05:00.0,548.8909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
x_train.drop("Unnamed: 9", axis=1, inplace=True)
x_test.drop("Unnamed: 9", axis=1, inplace=True)

In [197]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(417599, 9)
(417599, 5)
(226081, 9)


**Analysing na**

In [198]:
y_train.isna().sum()

time_step              0
washing_machine    10231
fridge_freezer     10231
TV                 10231
kettle             10231
dtype: int64

In [199]:
x_train.isna().sum()

time_step           0
consumption     10231
visibility     410663
temperature    410652
humidity       410663
humidex        410663
windchill      410671
wind           410663
pressure       410667
dtype: int64

Check if na values in global consumption reflect na values in the single appliances:

In [200]:
print((y_train["washing_machine"].isna() == x_train["consumption"].isna()).sum())
print((y_train["fridge_freezer"].isna() == x_train["consumption"].isna()).sum())
print((y_train["TV"].isna() == x_train["consumption"].isna()).sum())
print((y_train["kettle"].isna() == x_train["consumption"].isna()).sum())

417599
417599
417599
417599


Therefore, we can drop rows where we have no values for consumption:

In [201]:
x_train = x_train[x_train["consumption"].notna()]
y_train = y_train[y_train["TV"].notna()]

Final check:

In [202]:
(x_train.index == y_train.index).sum() == x_train.shape[0]

True

In [203]:
x_train

,time_step,consumption,visibility,temperature,humidity,humidex,windchill,wind,pressure
0,2013-03-17T00:01:00.0,550.4000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-03-17T00:02:00.0,548.6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-03-17T00:03:00.0,549.3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-03-17T00:04:00.0,549.3667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-03-17T00:05:00.0,548.8909,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
417594,2013-12-31T23:55:00.0,342.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417595,2013-12-31T23:56:00.0,311.5556,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417596,2013-12-31T23:57:00.0,310.5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417597,2013-12-31T23:58:00.0,312.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
x_train.describe()

,consumption,visibility,temperature,humidity,humidex,windchill,wind,pressure
count,407368.000000,6785.000000,6796.000000,6785.000000,6785.000000,6777.000000,6785.000000,6781.000000
mean,303.604240,23.941548,10.887184,82.860427,11.778276,9.371654,13.176566,1010.213774
std,343.624792,13.880512,4.690019,13.779871,5.477909,5.425497,7.931388,45.391857
min,106.000000,0.000000,-4.400000,26.000000,-4.400000,-5.900000,0.000000,1.000000
25%,158.000000,13.000000,7.700000,74.000000,7.700000,5.500000,7.000000,1005.600000
50%,192.000000,23.000000,10.900000,86.000000,11.600000,9.400000,11.000000,1013.300000
75%,312.250000,35.000000,14.100000,94.000000,15.700000,13.200000,19.000000,1019.300000
max,7571.222200,75.000000,27.800000,100.000000,30.700000,27.800000,52.000000,1039.500000


Looking at **time_step**:

In [205]:
x_train["time_step"] = pd.to_datetime(x_train["time_step"])

In [206]:
x_train.set_index("time_step", inplace=True)

Non-na values:

In [184]:
x_train.shape[0] - x_train.isna().sum() #/ x_train.shape[0]

time_step      407368
consumption    407368
visibility       6785
temperature      6796
humidity         6785
humidex          6785
windchill        6777
wind             6785
pressure         6781
dtype: int64

---

---

In [214]:
class DataImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        X = X.interpolate(method='linear').fillna(method='bfill')
        return X

In [215]:
di = DataImputer()
x_trans = di.transform(X=x_train)

**Baseline Metric**, fully connected network

In [209]:
class RNNDataFormatter(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        print(X.shape)
        nb_col = X.shape[1]
        print(nb_col)
        X_rnn = X[X.columns[:nb_col]].iloc[0:18000]
        X_rnn = X_rnn.values.reshape((int(X_rnn.shape[0]/60), 60, nb_col))
        return X_rnn

First Try:

In [216]:
x_trans.head()

,consumption,visibility,temperature,humidity,humidex,windchill,wind,pressure
time_step,,,,,,,,
2013-03-17 00:01:00,550.4000,35.0,8.9,86.0,8.9,6.0,19.0,1017.3
2013-03-17 00:02:00,548.6000,35.0,8.9,86.0,8.9,6.0,19.0,1017.3
2013-03-17 00:03:00,549.3000,35.0,8.9,86.0,8.9,6.0,19.0,1017.3
2013-03-17 00:04:00,549.3667,35.0,8.9,86.0,8.9,6.0,19.0,1017.3
2013-03-17 00:05:00,548.8909,35.0,8.9,86.0,8.9,6.0,19.0,1017.3


In [221]:
day = [g for n, g in x_trans.groupby(pd.Grouper(freq='D'))]

In [241]:
to_delete = [i for i in range(len(day)) if day[i].shape[0] != 1440]

for

In [228]:
for n, g in x_trans.groupby(pd.Grouper(freq='D')):
    

                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-03-17 00:01:00     550.4000        35.0     8.900000  86.000000   
2013-03-17 00:02:00     548.6000        35.0     8.900000  86.000000   
2013-03-17 00:03:00     549.3000        35.0     8.900000  86.000000   
2013-03-17 00:04:00     549.3667        35.0     8.900000  86.000000   
2013-03-17 00:05:00     548.8909        35.0     8.900000  86.000000   
...                          ...         ...          ...        ...   
2013-03-17 23:55:00     646.9000        30.0     7.316667  82.166667   
2013-03-17 23:56:00     646.4111        30.0     7.313333  82.133333   
2013-03-17 23:57:00     641.8556        30.0     7.310000  82.100000   
2013-03-17 23:58:00     638.9000        30.0     7.306667  82.066667   
2013-03-17 23:59:00     640.4000        30.0     7.303333  82.033333   

                      humidex  windchill       wind  pressure  

                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-04-20 00:00:00     139.8889   19.000000     4.800000  80.000000   
2013-04-20 00:01:00     139.7500   18.983333     4.803333  80.183333   
2013-04-20 00:02:00     140.5000   18.966667     4.806667  80.366667   
2013-04-20 00:03:00     156.5000   18.950000     4.810000  80.550000   
2013-04-20 00:04:00     178.3000   18.933333     4.813333  80.733333   
...                          ...         ...          ...        ...   
2013-04-20 23:55:00     137.8000   17.583333     7.100000  81.500000   
2013-04-20 23:56:00     138.8000   17.266667     7.060000  81.800000   
2013-04-20 23:57:00     185.2500   16.950000     7.020000  82.100000   
2013-04-20 23:58:00     176.2222   16.633333     6.980000  82.400000   
2013-04-20 23:59:00     170.6250   16.316667     6.940000  82.700000   

                      humidex  windchill       wind     pressur

[1439 rows x 8 columns]
                     consumption  visibility  temperature   humidity  humidex  \
time_step                                                                       
2013-05-15 00:00:00     136.1000   28.000000    11.400000  92.000000    12.70   
2013-05-15 00:01:00     146.9000   28.033333    11.398333  91.900000    12.69   
2013-05-15 00:02:00     172.3000   28.066667    11.396667  91.800000    12.68   
2013-05-15 00:03:00     167.2000   28.100000    11.395000  91.700000    12.67   
2013-05-15 00:04:00     166.3000   28.133333    11.393333  91.600000    12.66   
...                          ...         ...          ...        ...      ...   
2013-05-15 23:55:00     284.7000   34.583333    11.041667  82.916667    11.55   
2013-05-15 23:56:00     285.3333   34.666667    11.033333  82.933333    11.54   
2013-05-15 23:57:00     257.0000   34.750000    11.025000  82.950000    11.53   
2013-05-15 23:58:00     198.3000   34.833333    11.016667  82.966667    11.52   
2013

                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-06-21 00:00:00     180.2000   27.000000    13.100000  77.000000   
2013-06-21 00:01:00     180.5000   27.016667    13.101667  77.016667   
2013-06-21 00:02:00     183.0000   27.033333    13.103333  77.033333   
2013-06-21 00:03:00     184.2000   27.050000    13.105000  77.050000   
2013-06-21 00:04:00     181.4000   27.066667    13.106667  77.066667   
...                          ...         ...          ...        ...   
2013-06-21 23:55:00     756.2222   29.083333    12.975000  74.833333   
2013-06-21 23:56:00     858.7500   29.066667    12.960000  74.866667   
2013-06-21 23:57:00     587.3000   29.050000    12.945000  74.900000   
2013-06-21 23:58:00     413.9000   29.033333    12.930000  74.933333   
2013-06-21 23:59:00     455.4444   29.016667    12.915000  74.966667   

                       humidex  windchill       wind     pressu

[1439 rows x 8 columns]
                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-07-27 00:00:00     306.4000   35.000000    13.500000  88.000000   
2013-07-27 00:01:00     304.3000   35.083333    13.498333  87.966667   
2013-07-27 00:02:00     303.7000   35.166667    13.496667  87.933333   
2013-07-27 00:03:00     302.0000   35.250000    13.495000  87.900000   
2013-07-27 00:04:00     305.6000   35.333333    13.493333  87.866667   
...                          ...         ...          ...        ...   
2013-07-27 23:55:00     158.6667   10.416667    12.433333  99.000000   
2013-07-27 23:56:00     158.6667   10.333333    12.406667  99.000000   
2013-07-27 23:57:00     157.8750   10.250000    12.380000  99.000000   
2013-07-27 23:58:00     160.1818   10.166667    12.353333  99.000000   
2013-07-27 23:59:00     160.9167   10.083333    12.326667  99.000000   

                       humidex  windchi

...

                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-10-25 00:00:00     627.7500   45.000000     9.600000  80.000000   
2013-10-25 00:01:00     627.0000   44.916667     9.603333  79.983333   
2013-10-25 00:02:00     617.1111   44.833333     9.606667  79.966667   
2013-10-25 00:03:00     624.3889   44.750000     9.610000  79.950000   
2013-10-25 00:04:00     619.2889   44.666667     9.613333  79.933333   
...                          ...         ...          ...        ...   
2013-10-25 23:55:00     217.4444   13.416667    10.150000  87.166667   
2013-10-25 23:56:00     219.2500   12.933333    10.140000  87.333333   
2013-10-25 23:57:00     217.7778   12.450000    10.130000  87.500000   
2013-10-25 23:58:00     218.2222   11.966667    10.120000  87.666667   
2013-10-25 23:59:00     217.6250   11.483333    10.110000  87.833333   

                       humidex  windchill       wind     pressu

                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-11-04 00:00:00     161.1111   40.000000     4.100000  91.000000   
2013-11-04 00:01:00     160.6667   39.683333     4.091667  91.100000   
2013-11-04 00:02:00     160.0000   39.366667     4.083333  91.200000   
2013-11-04 00:03:00     240.4444   39.050000     4.075000  91.300000   
2013-11-04 00:04:00     258.6250   38.733333     4.066667  91.400000   
...                          ...         ...          ...        ...   
2013-11-04 23:55:00     153.8750   45.416667     6.625000  72.916667   
2013-11-04 23:56:00     153.4444   45.333333     6.620000  72.933333   
2013-11-04 23:57:00     153.2500   45.250000     6.615000  72.950000   
2013-11-04 23:58:00     155.1111   45.166667     6.610000  72.966667   
2013-11-04 23:59:00     161.3333   45.083333     6.605000  72.983333   

                      humidex  windchill       wind     pressur

[1438 rows x 8 columns]
                     consumption  visibility  temperature  humidity   humidex  \
time_step                                                                       
2013-12-01 00:00:00     186.0000    0.300000     3.800000     100.0  3.800000   
2013-12-01 00:01:00     187.0000    0.308333     3.788333     100.0  3.788333   
2013-12-01 00:02:00     185.8750    0.316667     3.776667     100.0  3.776667   
2013-12-01 00:03:00     185.3333    0.325000     3.765000     100.0  3.765000   
2013-12-01 00:04:00     215.6667    0.333333     3.753333     100.0  3.753333   
...                          ...         ...          ...       ...       ...   
2013-12-01 23:55:00     152.7778   45.416667     6.500000      85.0  6.500000   
2013-12-01 23:56:00     152.5000   45.333333     6.500000      85.0  6.500000   
2013-12-01 23:57:00     153.2222   45.250000     6.500000      85.0  6.500000   
2013-12-01 23:58:00     152.7778   45.166667     6.500000      85.0  6.500000   
2013

                     consumption  visibility  temperature  humidity   humidex  \
time_step                                                                       
2013-12-18 00:00:00     933.9556   40.000000     8.500000     95.00  8.700000   
2013-12-18 00:01:00     930.2222   39.533333     8.500000     95.05  8.703333   
2013-12-18 00:02:00     929.1111   39.066667     8.500000     95.10  8.706667   
2013-12-18 00:03:00     929.6667   38.600000     8.500000     95.15  8.710000   
2013-12-18 00:04:00     925.2500   38.133333     8.500000     95.20  8.713333   
...                          ...         ...          ...       ...       ...   
2013-12-18 10:24:00     155.6250   20.830769     6.292308     88.00  6.453846   
2013-12-18 10:25:00     156.6667   21.515385     6.221154     87.50  6.376923   
2013-12-18 10:26:00     156.4444   22.200000     6.150000     87.00  6.300000   
2013-12-18 10:27:00     157.3750   22.884615     6.078846     86.50  6.223077   
2013-12-18 10:28:00     165.

                     consumption  visibility  temperature   humidity  \
time_step                                                              
2013-12-28 00:00:00     159.2222   60.000000     0.400000  80.000000   
2013-12-28 00:01:00     159.7500   59.833333     0.401667  80.033333   
2013-12-28 00:02:00     159.1111   59.666667     0.403333  80.066667   
2013-12-28 00:03:00     159.0000   59.500000     0.405000  80.100000   
2013-12-28 00:04:00     159.8889   59.333333     0.406667  80.133333   
...                          ...         ...          ...        ...   
2013-12-28 23:55:00     312.7778    0.766667    -0.950000  99.916667   
2013-12-28 23:56:00     302.1111    0.693333    -0.920000  99.933333   
2013-12-28 23:57:00     305.3750    0.620000    -0.890000  99.950000   
2013-12-28 23:58:00     316.6667    0.546667    -0.860000  99.966667   
2013-12-28 23:59:00     318.5556    0.473333    -0.830000  99.983333   

                      humidex  windchill       wind     pressur

In [191]:
data = x_train.values.reshape((1, -1, 9))
target = y_train.iloc[:, 1].values.reshape(-1, 1)

ValueError: cannot reshape array of size 3258944 into shape (1,newaxis,9)

In [242]:
to_delete

[0,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 21,
 22,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 80,
 81,
 85,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 147,
 155]

In [108]:
X_train, Y_train = data[:, :7000, 1:], target[:7000, ]
X_valid, Y_valid = data[:, 7000:9000, 1:], target[7000:9000, :]
X_test, Y_test = data[:, 9000:, 1:], target[9000:, :]

In [109]:
print(f"X_train.shape is {X_train.shape}")
print(f"Y_train.shape is {Y_train.shape}")

X_train.shape is (1, 7000, 8)
Y_train.shape is (7000, 1)


In [70]:
Y_train.shape

(7000, 1)

In [68]:
Y_valid.shape

(2000, 1)

In [103]:
np.random.seed(42)
tf.random.set_seed(42)

model = Sequential([
    Flatten(input_shape=[7000, 8]),
    Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

ValueError: Input arrays should have the same number of samples as target arrays. Found 1 input samples and 7000 target samples.

TEST

In [111]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [112]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -1]

In [113]:
print(f"X_train.shape is {X_train.shape}")
print(f"Y_train.shape is {Y_train.shape}")
print()
print(f"X_valid.shape is {X_valid.shape}")
print(f"Y_valid.shape is {Y_valid.shape}")

X_train.shape is (7000, 50, 1)
Y_train.shape is (7000, 1)

X_valid.shape is (2000, 50, 1)
Y_valid.shape is (2000, 1)


In [97]:
np.random.seed(42)
tf.random.set_seed(42)

model = Sequential([
    Flatten(input_shape=[50, 1]),
    Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Train on 7000 samples, validate on 2000 samples
Epoch 1/20
7000/7000 [==============================] - 1s 133us/step - loss: 0.2206 - val_loss: 0.1020
Epoch 2/20
7000/7000 [==============================] - 1s 88us/step - loss: 0.0701 - val_loss: 0.0482
Epoch 3/20
7000/7000 [==============================] - 1s 80us/step - loss: 0.0356 - val_loss: 0.0257
Epoch 4/20
7000/7000 [==============================] - 1s 78us/step - loss: 0.0198 - val_loss: 0.0151
Epoch 5/20
7000/7000 [==============================] - 1s 79us/step - loss: 0.0125 - val_loss: 0.0104
Epoch 6/20
7000/7000 [==============================] - 1s 83us/step - loss: 0.0092 - val_loss: 0.0084
Epoch 7/20
7000/7000 [==============================] - 1s 80us/step - loss: 0.0077 - val_loss: 0.0074
Epoch 8/20
7000/7000 [==============================] - 1s 89us/step - loss: 0.0069 - val_loss: 0.0068
Epoch 9/20
7000/7000 [==============================] - 1s 84us/step - loss: 0.0063 - val_loss: 0.0062
Epoch 10/20
7000/7000 [=